### Research a generative, or selective model capable of proper retrieval from context
We saw that generation of an answer is not as straightforward as we thought it would be. Therefore, in this notebook I will compare different models on their performance on some hardcoded context. To see which one comes out on top.

In [43]:
import torch
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)
from pathlib import Path

Load the data:

In [44]:
import string

def remove_ws(text):
    return " ".join(text.split())

def remove_interpunct(text: str) -> str:
    """Remove all punctuation characters from the string."""
    return text.translate(str.maketrans("", "", string.punctuation))

def lowercase(text):
    return text.lower()

def normalize(text):
    text = remove_ws(text)
    #text = remove_interpunct(text)
    text = lowercase(text)
    return text

In [45]:

data_files = [f"{i}.txt" for i in range(1, 3)]  # list of data files I like as context
textual_context = ""                           # textual context to append to
base_dir = Path("./../data")                   # base dir where data is stored
for i, filename in enumerate(data_files):
    with open(base_dir.joinpath(filename), 'r') as file:
        text = file.read()
        text = normalize(text)                 # normalize text by removing ws, interpunct, and lowercase it
        textual_context += f"chunk {i}: {text}\n"          # add text from file to context
print(textual_context)

chunk 0: name: cream cake ingredients: fresh liquid cream: ¾cup(200 g) eggs: 3 type 00 flour: 1.6cups(200 g) sugar: 0.6cup(120 g) lemon peel: 1 baking powder: 3 ½tsp(16 g) salt: 1pinch powdered sugar: to taste instructions: cream cake, or torta alla panna, really holds a special place in the italian kitchen. why? because of its simplicity and authenticity. this classic cream cake recipe uses just a few basic ingredients—fresh cream, eggs and flour—leaving out butter or oil. the result? a moist and velvety cake. like, biting into a tender cloud. in italy, families enjoy this homemade cream cake throughout the day. in the morning, it might be dusted with powdered sugar. evening? it's dressed up with whipped cream and berries. whether you eat it plain or add chocolate shavings, this cake can make any moment feel more special. seriously good stuff. across different italian regions, people get creative with their easy cream cake . some add a hint of citrus zest for a unique twist. others th

### Define the tokenizer, model and pipeline:

In [49]:
gen_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # or your local/other LLaMA
tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
model = AutoModelForCausalLM.from_pretrained(gen_model_name)
pipeline = transformers.pipeline
pl = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=-1
)

Device set to use cpu


In [50]:
question = "What are the ingredients to egg white pancakes?"
prompt = f"Answer this question: {question} based on these chunks of context:\n"\
         f"{textual_context}"
out = pl(
    prompt,
    max_new_tokens=64, # max answer length
    num_beams=6,        # 4 candidates
    do_sample=False,    # deterministic
)[0]["generated_text"]
prediction_text = out.strip()
print(prediction_text)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Answer this question: What are the ingredients to egg white pancakes? based on these chunks of context:
chunk 0: name: cream cake ingredients: fresh liquid cream: ¾cup(200 g) eggs: 3 type 00 flour: 1.6cups(200 g) sugar: 0.6cup(120 g) lemon peel: 1 baking powder: 3 ½tsp(16 g) salt: 1pinch powdered sugar: to taste instructions: cream cake, or torta alla panna, really holds a special place in the italian kitchen. why? because of its simplicity and authenticity. this classic cream cake recipe uses just a few basic ingredients—fresh cream, eggs and flour—leaving out butter or oil. the result? a moist and velvety cake. like, biting into a tender cloud. in italy, families enjoy this homemade cream cake throughout the day. in the morning, it might be dusted with powdered sugar. evening? it's dressed up with whipped cream and berries. whether you eat it plain or add chocolate shavings, this cake can make any moment feel more special. seriously good stuff. across different italian regions, peopl